Install dependencies

In [ ]:
!pip install tensorboard
!pip install mltu
!pip install onnx
!pip install torch
!pip install torchsummaryX

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━

`model.py`
IAM dataset used

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def activation_layer(activation: str="relu", alpha: float=0.1, inplace: bool=True):
    """ Activation layer wrapper for LeakyReLU and ReLU activation functions

    Args:
        activation: str, activation function name (default: 'relu')
        alpha: float (LeakyReLU activation function parameter)

    Returns:
        torch.Tensor: activation layer
    """
    if activation == "relu":
        return nn.ReLU(inplace=inplace)

    elif activation == "leaky_relu":
        return nn.LeakyReLU(negative_slope=alpha, inplace=inplace)


class ConvBlock(nn.Module):
    """ Convolutional block with batch normalization
    """
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x: torch.Tensor):
        return self.bn(self.conv(x))


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_conv=True, stride=1, dropout=0.2, activation="leaky_relu"):
        super(ResidualBlock, self).__init__()
        self.convb1 = ConvBlock(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.act1 = activation_layer(activation)

        self.convb2 = ConvBlock(out_channels, out_channels, kernel_size=3, stride=1, padding=1)

        self.dropout = nn.Dropout(p=dropout)

        self.shortcut = None
        if skip_conv:
            if stride != 1 or in_channels != out_channels:
                self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)

        self.act2 = activation_layer(activation)

    def forward(self, x):
        skip = x

        out = self.act1(self.convb1(x))
        out = self.convb2(out)

        if self.shortcut is not None:
            out += self.shortcut(skip)

        out = self.act2(out)
        out = self.dropout(out)

        return out

class Network(nn.Module):
    """ Handwriting recognition network for CTC loss"""
    def __init__(self, num_chars: int, activation: str="leaky_relu", dropout: float=0.2):
        super(Network, self).__init__()

        self.rb1 = ResidualBlock(3, 16, skip_conv = True, stride=1, activation=activation, dropout=dropout)
        self.rb2 = ResidualBlock(16, 16, skip_conv = True, stride=2, activation=activation, dropout=dropout)
        self.rb3 = ResidualBlock(16, 16, skip_conv = False, stride=1, activation=activation, dropout=dropout)

        self.rb4 = ResidualBlock(16, 32, skip_conv = True, stride=2, activation=activation, dropout=dropout)
        self.rb5 = ResidualBlock(32, 32, skip_conv = False, stride=1, activation=activation, dropout=dropout)

        self.rb6 = ResidualBlock(32, 64, skip_conv = True, stride=2, activation=activation, dropout=dropout)
        self.rb7 = ResidualBlock(64, 64, skip_conv = True, stride=1, activation=activation, dropout=dropout)

        self.rb8 = ResidualBlock(64, 64, skip_conv = False, stride=1, activation=activation, dropout=dropout)
        self.rb9 = ResidualBlock(64, 64, skip_conv = False, stride=1, activation=activation, dropout=dropout)

        self.lstm = nn.LSTM(64, 128, bidirectional=True, num_layers=1, batch_first=True)
        self.lstm_dropout = nn.Dropout(p=dropout)

        self.output = nn.Linear(256, num_chars + 1)

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        # normalize images between 0 and 1
        images_flaot = images / 255.0

        # transpose image to channel first
        images_flaot = images_flaot.permute(0, 3, 1, 2)

        # apply convolutions
        x = self.rb1(images_flaot)
        x = self.rb2(x)
        x = self.rb3(x)
        x = self.rb4(x)
        x = self.rb5(x)
        x = self.rb6(x)
        x = self.rb7(x)
        x = self.rb8(x)
        x = self.rb9(x)

        x = x.reshape(x.size(0), -1, x.size(1))

        x, _ = self.lstm(x)
        x = self.lstm_dropout(x)

        x = self.output(x)
        x = F.log_softmax(x, 2)

        return x

`configs.py`

In [ ]:
import os
from datetime import datetime

from mltu.configs import BaseModelConfigs


class ModelConfigs(BaseModelConfigs):
  def __init__(self):
    super().__init__()
    self.model_path = os.path.join("Models/IAM_Training", datetime.strftime(datetime.now(), "%Y%m%d%H%M"))
    self.vocab = ""
    self.height = 32
    self.width = 128
    self.max_text_length = 0
    self.batch_size = 64
    self.learning_rate = 0.002
    self.train_epochs = 1000

Importing modules

In [ ]:
import os
import tarfile
from tqdm import tqdm
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import torch
import torch.optim as optim
from torchsummaryX import summary

from mltu.torch.model import Model
from mltu.torch.losses import CTCLoss
from mltu.torch.dataProvider import DataProvider
from mltu.torch.metrics import CERMetric, WERMetric
from mltu.torch.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Model2onnx, ReduceLROnPlateau

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding, ImageShowCV2
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate, RandomSharpen
from mltu.annotations.images import CVImage

# from model import Network
# from configs import ModelConfigs

Download and extract dataset

In [ ]:
def download_and_unzip(url, extract_to='Datasets', chunk_size=1024*1024):
  http_response = urlopen(url)

  data = b''
  iterations = http_response.length // chunk_size + 1
  for _ in tqdm(range(iterations)):
    data += http_response.read(chunk_size)

  zipfile = ZipFile(BytesIO(data))
  zipfile.extractall(path=extract_to)

dataset_path = os.path.join('Datasets', 'IAM_Words')
if not os.path.exists(dataset_path):
  download_and_unzip('https://git.io/J0fjL', extract_to='Datasets')

  file = tarfile.open(os.path.join(dataset_path, "words.tgz"))
  file.extractall(os.path.join(dataset_path, "words"))

100%|██████████| 784/784 [04:43<00:00,  2.76it/s]


Data preprocessing

This piece of code performs data preprocessing by parsing a `words.txt` file and populating three variables: `dataset`, `vocab`, and `max_len`. The dataset is a list that contains lists. Each inner list comprises a file path and its label. The vocab is a set of unique characters present in the labels. The `max_len` is the maximum length of the labels.

For each line in the file, the code executes the following tasks:

1. Skips the line if it starts with #;
2. Skips the line if the second element after splitting the line by space is "err";
3. Extracts the first three and eight characters of the filename and label, respectively;
4. Joins the dataset_path with the extracted folder names and filenames to form the file path;
5. Skips the line if the file path does not exist;
6. Otherwise, it adds the file path and `label` to the dataset list. Additionally, it updates the vocab set with the characters present in the label and updates the `max_len` variable to hold the maximum value of the current `max_len` and the length of the label;
After pre-processing the dataset, the code saves the vocabulary and the maximum text length to configs using the `ModelConfigs` class.




In [ ]:
dataset, vocab, max_len = [], set(), 0

# Preprocess the dataset by the specific IAM_Words dataset file structure
words = open(os.path.join(dataset_path, "words.txt"), "r").readlines()
for line in tqdm(words):
  if line.startswith("#"):
    continue

  line_split = line.split(" ")
  if line_split[1] == "err":
    continue

  folder1 = line_split[0][:3]
  folder2 = "-".join(line_split[0].split("-")[:2])
  file_name = line_split[0] + ".png"
  label = line_split[-1].rstrip('\n')

  rel_path = os.path.join(dataset_path, "words", folder1, folder2, file_name)
  if not os.path.exists(rel_path):
    print(f"File not found: {rel_path}")
    continue

  dataset.append([rel_path, label])
  vocab.update(list(label))
  max_len = max(max_len, len(label))

configs = ModelConfigs()

# Save vocab and maximum text length to configs
configs.vocab = "".join(sorted(vocab))
configs.max_text_length = max_len
configs.save()


100%|██████████| 115338/115338 [00:01<00:00, 65412.15it/s]


In [ ]:
!head -50 "Datasets/IAM_Words/words.txt"

#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
a01-000u-00-00 ok 154 408 768 27 51 AT A
a01-000u-00-01 ok 154 507 766 213 48 NN MOVE
a01-000u-00-02 ok 154 796 764 70 50 TO to
a01-000u-00-03 ok 154 919 757 166 78 VB stop
a01-000u-00-04 ok 154 118

Prepare the data provider using `DataProvider` class

In [ ]:
# Create a data provider for the dataset
data_provider = DataProvider(
  dataset=dataset,
  skip_validation=True,
  batch_size=configs.batch_size,
  data_preprocessors=[ImageReader(CVImage)],
  transformers=[
    #ImageShowCV2(), # uncomment to show images during training
    ImageResizer(configs.width, configs.height, keep_aspect_ratio=False),
    LabelIndexer(configs.vocab),
    LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab))
    ],
  use_cache=True,
)

Split datasets into training and validation sets

In [ ]:
train_dataProvider, test_dataProvider = data_provider.split(split = 0.9)

Add a few augmentation techniques to our training data provider that help us to train a better model:


In [ ]:
train_dataProvider.augmentors = [
    RandomBrightness(),
    RandomErodeDilate(),
    RandomSharpen(),
    RandomRotate(angle=10),
    ]

## Model

In [ ]:
network = Network(len(configs.vocab), activation='leaky_relu', dropout=0.3)


We create an `optim.Adam` object for the optimizer and an `CTCLoss` object for the loss function. The `CTCLoss` is used because the problem being solved is a character recognition problem where the length of the input and output sequence is not necessarily the same:



In [ ]:
loss = CTCLoss(blank=len(configs.vocab))
optimizer = optim.Adam(network.parameters(), lr=configs.learning_rate)

In [ ]:
#!pip install pandas==1.5.2

In [ ]:
# torchsummaryX is not fully updated here
# error is solved in github issues (https://github.com/nmhkahn/torchsummaryX/pull/21/commits/c1d32c4e4f3044ac1724d6ee4b4c2d080c60cfe0)

#summary(network, torch.zeros((1, configs.height, configs.width, 3)))
'''
===================================================================================
                                Kernel Shape      Output Shape    Params  \
Layer
0_rb1.convb1.Conv2d_conv       [3, 16, 3, 3]  [1, 16, 32, 128]     448.0
1_rb1.convb1.BatchNorm2d_bn             [16]  [1, 16, 32, 128]      32.0
2_rb1.LeakyReLU_act1                       -  [1, 16, 32, 128]         -
3_rb1.convb2.Conv2d_conv      [16, 16, 3, 3]  [1, 16, 32, 128]     2.32k
4_rb1.convb2.BatchNorm2d_bn             [16]  [1, 16, 32, 128]      32.0
5_rb1.Conv2d_shortcut          [3, 16, 1, 1]  [1, 16, 32, 128]      64.0
6_rb1.LeakyReLU_act2                       -  [1, 16, 32, 128]         -
7_rb1.Dropout_dropout                      -  [1, 16, 32, 128]         -
8_rb2.convb1.Conv2d_conv      [16, 16, 3, 3]   [1, 16, 16, 64]     2.32k
9_rb2.convb1.BatchNorm2d_bn             [16]   [1, 16, 16, 64]      32.0
10_rb2.LeakyReLU_act1                      -   [1, 16, 16, 64]         -
11_rb2.convb2.Conv2d_conv     [16, 16, 3, 3]   [1, 16, 16, 64]     2.32k
12_rb2.convb2.BatchNorm2d_bn            [16]   [1, 16, 16, 64]      32.0
13_rb2.Conv2d_shortcut        [16, 16, 1, 1]   [1, 16, 16, 64]     272.0
14_rb2.LeakyReLU_act2                      -   [1, 16, 16, 64]         -
15_rb2.Dropout_dropout                     -   [1, 16, 16, 64]         -
16_rb3.convb1.Conv2d_conv     [16, 16, 3, 3]   [1, 16, 16, 64]     2.32k
17_rb3.convb1.BatchNorm2d_bn            [16]   [1, 16, 16, 64]      32.0
18_rb3.LeakyReLU_act1                      -   [1, 16, 16, 64]         -
19_rb3.convb2.Conv2d_conv     [16, 16, 3, 3]   [1, 16, 16, 64]     2.32k
20_rb3.convb2.BatchNorm2d_bn            [16]   [1, 16, 16, 64]      32.0
21_rb3.LeakyReLU_act2                      -   [1, 16, 16, 64]         -
22_rb3.Dropout_dropout                     -   [1, 16, 16, 64]         -
23_rb4.convb1.Conv2d_conv     [16, 32, 3, 3]    [1, 32, 8, 32]     4.64k
24_rb4.convb1.BatchNorm2d_bn            [32]    [1, 32, 8, 32]      64.0
25_rb4.LeakyReLU_act1                      -    [1, 32, 8, 32]         -
26_rb4.convb2.Conv2d_conv     [32, 32, 3, 3]    [1, 32, 8, 32]    9.248k
27_rb4.convb2.BatchNorm2d_bn            [32]    [1, 32, 8, 32]      64.0
28_rb4.Conv2d_shortcut        [16, 32, 1, 1]    [1, 32, 8, 32]     544.0
29_rb4.LeakyReLU_act2                      -    [1, 32, 8, 32]         -
30_rb4.Dropout_dropout                     -    [1, 32, 8, 32]         -
31_rb5.convb1.Conv2d_conv     [32, 32, 3, 3]    [1, 32, 8, 32]    9.248k
32_rb5.convb1.BatchNorm2d_bn            [32]    [1, 32, 8, 32]      64.0
33_rb5.LeakyReLU_act1                      -    [1, 32, 8, 32]         -
34_rb5.convb2.Conv2d_conv     [32, 32, 3, 3]    [1, 32, 8, 32]    9.248k
35_rb5.convb2.BatchNorm2d_bn            [32]    [1, 32, 8, 32]      64.0
36_rb5.LeakyReLU_act2                      -    [1, 32, 8, 32]         -
37_rb5.Dropout_dropout                     -    [1, 32, 8, 32]         -
38_rb6.convb1.Conv2d_conv     [32, 64, 3, 3]    [1, 64, 4, 16]   18.496k
39_rb6.convb1.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
40_rb6.LeakyReLU_act1                      -    [1, 64, 4, 16]         -
41_rb6.convb2.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
42_rb6.convb2.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
43_rb6.Conv2d_shortcut        [32, 64, 1, 1]    [1, 64, 4, 16]    2.112k
44_rb6.LeakyReLU_act2                      -    [1, 64, 4, 16]         -
45_rb6.Dropout_dropout                     -    [1, 64, 4, 16]         -
46_rb7.convb1.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
47_rb7.convb1.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
48_rb7.LeakyReLU_act1                      -    [1, 64, 4, 16]         -
49_rb7.convb2.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
50_rb7.convb2.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
51_rb7.LeakyReLU_act2                      -    [1, 64, 4, 16]         -
52_rb7.Dropout_dropout                     -    [1, 64, 4, 16]         -
53_rb8.convb1.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
54_rb8.convb1.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
55_rb8.LeakyReLU_act1                      -    [1, 64, 4, 16]         -
56_rb8.convb2.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
57_rb8.convb2.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
58_rb8.LeakyReLU_act2                      -    [1, 64, 4, 16]         -
59_rb8.Dropout_dropout                     -    [1, 64, 4, 16]         -
60_rb9.convb1.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
61_rb9.convb1.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
62_rb9.LeakyReLU_act1                      -    [1, 64, 4, 16]         -
63_rb9.convb2.Conv2d_conv     [64, 64, 3, 3]    [1, 64, 4, 16]   36.928k
64_rb9.convb2.BatchNorm2d_bn            [64]    [1, 64, 4, 16]     128.0
65_rb9.LeakyReLU_act2                      -    [1, 64, 4, 16]         -
66_rb9.Dropout_dropout                     -    [1, 64, 4, 16]         -
67_lstm                                    -      [1, 64, 256]  198.656k
68_lstm_dropout                            -      [1, 64, 256]         -
69_output                          [256, 79]       [1, 64, 79]   20.303k

                              Mult-Adds
Layer
0_rb1.convb1.Conv2d_conv      1.769472M
1_rb1.convb1.BatchNorm2d_bn        16.0
2_rb1.LeakyReLU_act1                  -
3_rb1.convb2.Conv2d_conv      9.437184M
4_rb1.convb2.BatchNorm2d_bn        16.0
5_rb1.Conv2d_shortcut          196.608k
6_rb1.LeakyReLU_act2                  -
7_rb1.Dropout_dropout                 -
8_rb2.convb1.Conv2d_conv      2.359296M
9_rb2.convb1.BatchNorm2d_bn        16.0
10_rb2.LeakyReLU_act1                 -
11_rb2.convb2.Conv2d_conv     2.359296M
12_rb2.convb2.BatchNorm2d_bn       16.0
13_rb2.Conv2d_shortcut         262.144k
14_rb2.LeakyReLU_act2                 -
15_rb2.Dropout_dropout                -
16_rb3.convb1.Conv2d_conv     2.359296M
17_rb3.convb1.BatchNorm2d_bn       16.0
18_rb3.LeakyReLU_act1                 -
19_rb3.convb2.Conv2d_conv     2.359296M
20_rb3.convb2.BatchNorm2d_bn       16.0
21_rb3.LeakyReLU_act2                 -
22_rb3.Dropout_dropout                -
23_rb4.convb1.Conv2d_conv     1.179648M
24_rb4.convb1.BatchNorm2d_bn       32.0
25_rb4.LeakyReLU_act1                 -
26_rb4.convb2.Conv2d_conv     2.359296M
27_rb4.convb2.BatchNorm2d_bn       32.0
28_rb4.Conv2d_shortcut         131.072k
29_rb4.LeakyReLU_act2                 -
30_rb4.Dropout_dropout                -
31_rb5.convb1.Conv2d_conv     2.359296M
32_rb5.convb1.BatchNorm2d_bn       32.0
33_rb5.LeakyReLU_act1                 -
34_rb5.convb2.Conv2d_conv     2.359296M
35_rb5.convb2.BatchNorm2d_bn       32.0
36_rb5.LeakyReLU_act2                 -
37_rb5.Dropout_dropout                -
38_rb6.convb1.Conv2d_conv     1.179648M
39_rb6.convb1.BatchNorm2d_bn       64.0
40_rb6.LeakyReLU_act1                 -
41_rb6.convb2.Conv2d_conv     2.359296M
42_rb6.convb2.BatchNorm2d_bn       64.0
43_rb6.Conv2d_shortcut         131.072k
44_rb6.LeakyReLU_act2                 -
45_rb6.Dropout_dropout                -
46_rb7.convb1.Conv2d_conv     2.359296M
47_rb7.convb1.BatchNorm2d_bn       64.0
48_rb7.LeakyReLU_act1                 -
49_rb7.convb2.Conv2d_conv     2.359296M
50_rb7.convb2.BatchNorm2d_bn       64.0
51_rb7.LeakyReLU_act2                 -
52_rb7.Dropout_dropout                -
53_rb8.convb1.Conv2d_conv     2.359296M
54_rb8.convb1.BatchNorm2d_bn       64.0
55_rb8.LeakyReLU_act1                 -
56_rb8.convb2.Conv2d_conv     2.359296M
57_rb8.convb2.BatchNorm2d_bn       64.0
58_rb8.LeakyReLU_act2                 -
59_rb8.Dropout_dropout                -
60_rb9.convb1.Conv2d_conv     2.359296M
61_rb9.convb1.BatchNorm2d_bn       64.0
62_rb9.LeakyReLU_act1                 -
63_rb9.convb2.Conv2d_conv     2.359296M
64_rb9.convb2.BatchNorm2d_bn       64.0
65_rb9.LeakyReLU_act2                 -
66_rb9.Dropout_dropout                -
67_lstm                        196.608k
68_lstm_dropout                       -
69_output                       20.224k
-----------------------------------------------------------------------------------
                         Totals
Total params           544.847k
Trainable params       544.847k
Non-trainable params        0.0
Mult-Adds             47.53456M
===================================================================================
'''

In [ ]:
# Put on cuda if available
if torch.cuda.is_available():
    network = network.cuda()
    print("CUDA available: CUDA to be used")

The Model class takes four main arguments: `network`, `optimizer`, `loss`, and `metrics`. The Network is an instance of a PyTorch neural network model that takes input tensors and returns output tensors. `Optimizer` is an instance of a PyTorch optimizer that takes the network's parameters as input and updates them based on the gradients computed during backpropagation. `Loss` is an instance of a PyTorch loss function that computes the difference between the network's predictions and the actual labels. `Metrics` is a list of instances of custom metrics that evaluate the performance of the network on the validation set during training.

Th`e Mod`el class has several methods for training and testing the model. Th`e f`it method trains the model for a specified number of epochs using the training and validation data providers and applies the specified callbacks at each training stage. The evaluation method evaluates the performance of the model on a given dataset using the specified metrics.

In the code, the Model object is instantiated with the Neural Network instance, the Adam optimizer, the CTCLoss loss function loss, and a list of two custom metri`cs: CERMe`tric `and WERMe`tric. CERMetric computes the Character Error Rate (CER), which is the ratio of the number of incorrect characters to the total number of characters in the predictions. WERMetric computes the Word Error Rate (WER), which is the ratio of the number of incorrectly recognized words to the total number of words in the predictionFinally, the Model instance's fit method is called with the `train_dataProvider` and `test_dataProvider` datasets, the number of epochs set to 1000, and a list of five callbacks: `earlyStopping`, `modelCheckpoint`, `tb_callback`, `reduce_lr`, and `model2onnx`. These callbacks are defined in the `mltu.torch.callbacks` module and are used to save the best model based on validation loss, stop the training process early if the model's performance does not improve for a specified number of epochs, log the model's performance metrics using TensorBoard, reduce the learning rate when the model's performance plateaus, and save the model in the  ONNX format for deployment.

First try model had acc. of ~75% on training dataset(50 epochs).

For second try, epochs are reduced to 500.



In [ ]:
# create callbacks
earlyStopping = EarlyStopping(monitor='val_CER', patience=20, mode="min", verbose=1)
modelCheckpoint = ModelCheckpoint(configs.model_path + '/model.pt', monitor='val_CER', mode="min", save_best_only=True, verbose=1)
tb_callback = TensorBoard(configs.model_path + '/logs')
reduce_lr = ReduceLROnPlateau(monitor='val_CER', factor=0.9, patience=10, verbose=1, mode='min', min_lr=1e-6)
model2onnx = Model2onnx(
    saved_model_path=configs.model_path + '/model.pt',
    input_shape=(1, configs.height, configs.width, 3),
    verbose=1,
    metadata={"vocab": configs.vocab}
    )

# create model object that will handle training and testing of the network
model = Model(network, optimizer, loss, metrics=[CERMetric(configs.vocab), WERMetric(configs.vocab)])
model.fit(
    train_dataProvider,
    test_dataProvider,
    epochs=200,
    callbacks=[earlyStopping, modelCheckpoint, tb_callback, reduce_lr, model2onnx]
    )

# Save training and validation datasets as csv files
train_dataProvider.to_csv(os.path.join(configs.model_path, 'train.csv'))
test_dataProvider.to_csv(os.path.join(configs.model_path, 'val.csv'))


For testing purposes, validation list in `csv` and model in `.onnx` format needed

In [ ]:
# Import mltu, numpy libraries

import cv2
import typing
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shapes[0][1:3][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(self.output_names, {self.input_names[0]: image_pred})[0]

        text = ctc_decoder(preds, self.metadata["vocab"])[0]

        return text

if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm

    model = ImageToWordModel(model_path="/content/model.onnx")

    df = pd.read_csv("/content/val.csv").values.tolist()

    ''' For single prediction

    image_path = "C:/Users/Shyam/Downloads/create.jpg"
    image = cv2.imread(image_path.replace("\\", "/"))

    prediction_text = model.predict(image)
    print(f"Prediction: {prediction_text}")
    '''

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path)

        prediction_text = model.predict(image)

        cer = get_cer(prediction_text, label)
        print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")

  0%|          | 19/9646 [00:00<01:35, 101.19it/s]

Image: Datasets/IAM_Words/words/p03/p03-004/p03-004-00-03.png, Label: sat, Prediction: sat, CER: 0.0
Image: Datasets/IAM_Words/words/m04/m04-030/m04-030-03-07.png, Label: ., Prediction: ., CER: 0.0
Image: Datasets/IAM_Words/words/b04/b04-054/b04-054-03-05.png, Label: Cypriots, Prediction: Eypriots, CER: 0.125
Image: Datasets/IAM_Words/words/n06/n06-128/n06-128-02-01.png, Label: turned, Prediction: turned, CER: 0.0
Image: Datasets/IAM_Words/words/m06/m06-067/m06-067-03-04.png, Label: ', Prediction: ,, CER: 1.0
Image: Datasets/IAM_Words/words/p06/p06-058/p06-058-05-04.png, Label: ,, Prediction: ,, CER: 0.0
Image: Datasets/IAM_Words/words/g06/g06-011i/g06-011i-01-03.png, Label: that, Prediction: that, CER: 0.0
Image: Datasets/IAM_Words/words/c06/c06-058/c06-058-04-02.png, Label: a, Prediction: a, CER: 0.0
Image: Datasets/IAM_Words/words/g06/g06-037k/g06-037k-00-06.png, Label: I, Prediction: ,, CER: 1.0
Image: Datasets/IAM_Words/words/e02/e02-086/e02-086-06-04.png, Label: will, Prediction:

  0%|          | 45/9646 [00:00<01:23, 115.11it/s]

Image: Datasets/IAM_Words/words/h07/h07-057a/h07-057a-04-01.png, Label: year, Prediction: year, CER: 0.0
Image: Datasets/IAM_Words/words/h04/h04-000/h04-000-03-04.png, Label: incomplete, Prediction: mnuglete, CER: 0.5
Image: Datasets/IAM_Words/words/l04/l04-153/l04-153-06-05.png, Label: that, Prediction: that, CER: 0.0
Image: Datasets/IAM_Words/words/d06/d06-091/d06-091-02-09.png, Label: Save, Prediction: Save, CER: 0.0
Image: Datasets/IAM_Words/words/b06/b06-093/b06-093-09-05.png, Label: whether, Prediction: wether, CER: 0.14285714285714285
Image: Datasets/IAM_Words/words/a01/a01-096u/a01-096u-02-05.png, Label: had, Prediction: had, CER: 0.0
Image: Datasets/IAM_Words/words/j06/j06-056/j06-056-01-01.png, Label: light, Prediction: lighut, CER: 0.2
Image: Datasets/IAM_Words/words/b04/b04-128/b04-128-02-07.png, Label: cheered, Prediction: chened, CER: 0.2857142857142857
Image: Datasets/IAM_Words/words/g07/g07-069a/g07-069a-02-01.png, Label: to, Prediction: to, CER: 0.0
Image: Datasets/IAM

  1%|          | 69/9646 [00:00<01:26, 110.93it/s]

...

Image: Datasets/IAM_Words/words/c02/c02-056/c02-056-06-10.png, Label: which, Prediction: which, CER: 0.0
Image: Datasets/IAM_Words/words/h06/h06-000/h06-000-06-03.png, Label: types, Prediction: types, CER: 0.0
Image: Datasets/IAM_Words/words/b04/b04-081/b04-081-05-00.png, Label: in, Prediction: in, CER: 0.0
Image: Datasets/IAM_Words/words/a02/a02-008/a02-008-07-06.png, Label: advise, Prediction: advise, CER: 0.0
Image: Datasets/IAM_Words/words/b04/b04-047/b04-047-05-03.png, Label: of, Prediction: of, CER: 0.0
Image: Datasets/IAM_Words/words/e01/e01-035/e01-035-03-03.png, Label: to, Prediction: to, CER: 0.0
Image: Datasets/IAM_Words/words/p02/p02-081/p02-081-05-05.png, Label: flirting, Prediction: fiting, CER: 0.25
Image: Datasets/IAM_Words/words/c06/c06-087/c06-087-03-10.png, Label: an, Prediction: an, CER: 0.0
Image: Datasets/IAM_Words/words/a02/a02-072/a02-072-00-08.png, Label: be, Prediction: be, CER: 0.0
Image: Datasets/IAM_Words/words/h07/h07-066b/h07-066b-01-05.png, Label: are, P

100%|█████████▉| 9626/9646 [01:01<00:00, 171.87it/s]

Image: Datasets/IAM_Words/words/h07/h07-069a/h07-069a-00-07.png, Label: settle, Prediction: settle, CER: 0.0
Image: Datasets/IAM_Words/words/d04/d04-089/d04-089-03-01.png, Label: learn, Prediction: learn, CER: 0.0
Image: Datasets/IAM_Words/words/c03/c03-081a/c03-081a-02-06.png, Label: altogether, Prediction: altogether, CER: 0.0
Image: Datasets/IAM_Words/words/h04/h04-025/h04-025-03-03.png, Label: -2.6, Prediction: 1.8, CER: 0.75
Image: Datasets/IAM_Words/words/c03/c03-021a/c03-021a-07-01.png, Label: his, Prediction: his, CER: 0.0
Image: Datasets/IAM_Words/words/e07/e07-076/e07-076-00-05.png, Label: clay, Prediction: clay, CER: 0.0
Image: Datasets/IAM_Words/words/c04/c04-017/c04-017-06-02.png, Label: ,, Prediction: I, CER: 1.0
Image: Datasets/IAM_Words/words/e02/e02-094/e02-094-01-01.png, Label: ,, Prediction: ,, CER: 0.0
Image: Datasets/IAM_Words/words/l07/l07-074/l07-074-05-00.png, Label: it, Prediction: it, CER: 0.0
Image: Datasets/IAM_Words/words/g06/g06-045a/g06-045a-02-08.png, La

100%|██████████| 9646/9646 [01:01<00:00, 155.77it/s]

Image: Datasets/IAM_Words/words/n06/n06-119/n06-119-04-07.png, Label: lay, Prediction: lay, CER: 0.0
Image: Datasets/IAM_Words/words/g06/g06-042m/g06-042m-01-06.png, Label: has, Prediction: has, CER: 0.0
Image: Datasets/IAM_Words/words/e04/e04-095/e04-095-00-02.png, Label: of, Prediction: of, CER: 0.0
Image: Datasets/IAM_Words/words/b04/b04-134/b04-134-04-04.png, Label: -, Prediction: -, CER: 0.0
Image: Datasets/IAM_Words/words/e04/e04-086/e04-086-03-00.png, Label: ash, Prediction: ask, CER: 0.3333333333333333
Image: Datasets/IAM_Words/words/a06/a06-095/a06-095-10-02.png, Label: ., Prediction: ., CER: 0.0
Image: Datasets/IAM_Words/words/c02/c02-000/c02-000-06-01.png, Label: Antonioni's, Prediction: antmiani's, CER: 0.36363636363636365
Average CER: 0.09479855259773178


In [ ]:
# For single prediction
from google.colab.patches import cv2_imshow
image_path = "/content/sig.PNG"
#image_path = "/content/Datasets/IAM_Words/words/a02/a02-004/a02-004-02-00.png"
image = cv2.imread(image_path.replace("\\", "/"))

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert image to greyscale
image = np.stack((image,) * 3, axis=-1) # Expand dims for user input images
print(image.shape)
#cv2_imshow(image)

prediction_text = model.predict(image)
print(f"Prediction: {prediction_text}")

(137, 380, 3)
Prediction: srn
